# Evaluación comparativa de la transposición de ejes
Este cuaderno compara el rendimiento de diferentes implementaciones de transposición de ejes.

**Nota:** los resultados de la evaluación comparativa varían considerablemente dependiendo del tamaño de la imagen, el tamaño del kernel, las operaciones utilizadas, los parámetros y el hardware empleado. Utiliza este cuaderno para adaptarlo a tu escenario de uso y realizar la evaluación comparativa en tu hardware objetivo. Si tienes diferentes escenarios o casos de uso, ¡eres bienvenido a enviar tu cuaderno como una solicitud de extracción!

In [1]:
import pyclesperanto_prototype as cle
import numpy as np
import time
import cupy as cp

# para medir correctamente la duración de la ejecución del kernel, necesitamos establecer esta bandera. Sin embargo, ralentizará un poco la ejecución de los flujos de trabajo
cle.set_wait_for_kernel_finish(True)

# selecciona una GPU con el siguiente nombre. Esto recurrirá a cualquier otra GPU si no se encuentra ninguna con este nombre
cle.select_device('RTX')

<NVIDIA GeForce RTX 3050 Ti Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [2]:
# datos de prueba
import numpy as np

test_image = np.random.random([100, 512, 1024])

## clEsperanto

In [3]:
# transposición con pyclesperanto
result_image = None

test_image_gpu = cle.push_zyx(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cle.transpose_xz(test_image_gpu, result_image)
    print("duración de la transposición de pyclesperanto: " + str(time.time() - start_time))
print(result_image.shape)

pyclesperanto transpose duration: 0.06201291084289551
pyclesperanto transpose duration: 0.04100918769836426
pyclesperanto transpose duration: 0.040008544921875
pyclesperanto transpose duration: 0.040008544921875
pyclesperanto transpose duration: 0.0400090217590332
pyclesperanto transpose duration: 0.04129624366760254
pyclesperanto transpose duration: 0.042009592056274414
pyclesperanto transpose duration: 0.04128861427307129
pyclesperanto transpose duration: 0.04102063179016113
pyclesperanto transpose duration: 0.04099869728088379
(1024, 512, 100)


## cupy

In [4]:
# transposición con numpy
result_image = None
cu_test_image = cp.asarray(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cp.transpose(cu_test_image, (2, 1, 0))
    cp.cuda.stream.get_current_stream().synchronize() # necesitamos esperar aquí para medir el tiempo correctamente
    print("duración de la transposición de cupy: " + str(time.time() - start_time))
print(result_image.shape)

cupy transpose duration: 0.06301379203796387
cupy transpose duration: 0.0009999275207519531
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
(1024, 512, 100)


## numpy

In [5]:
# transposición con numpy
result_image = None

for i in range(0, 10):
    start_time = time.time()
    result_image = np.transpose(test_image, (2, 1, 0))
    print("duración de la transposición de numpy: " + str(time.time() - start_time))
print(result_image.shape)

numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
(1024, 512, 100)
